# 딥 러닝을 이용한 자연어 처리 입문

아래 링크의 E-book을 보고 실습한 내용입니다.

WikiDocs 주소: https://wikidocs.net/31766


# 2장 텍스트 전처리

## 6절 정수 인코딩 실습


## 정수 인코딩

컴퓨터는 텍스트보다는 숫자를 더 잘 처리할 수 있기 때문에 자연어 처리를 할 때도 텍스트를 숫자로 바꾸는 것이 좋다. 정수 인코딩은 텍스트를 숫자로 바꾸는 기법을 적용하기 전에 각 단어를 고유한 정수로 맵핑 시키는 전처리 작업이다.

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### dictonary를 사용하여 구현하기

In [3]:
text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

# 문장 토큰화
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [4]:
# 불용어를 제거하고 vocab에 단어 빈도수를 저장한다.
vocab = {}
sentences = []
stop_words = set(stopwords.words("english"))

for i in text:
    sentence = word_tokenize(i)
    result = []
    
    for word in sentence:
        word = word.lower()
        if word not in stop_words:
            if len(word) > 2:
                result.append(word)
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1
    sentences.append(result)
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [5]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [6]:
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [7]:
# 정제 작업 - 1번만 나온 단어들은 제거해준다.
word_to_index = {}
i=0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : 
        i=i+1
        word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [8]:
# 빈도수가 높은 상위 5개의 단어만 저장한다.
vocab_size = 5
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제

# 빈도수가 낮은 단어는 제외했기 때문에 문장을 분석할 때 vocab에 없는 단어가 나올 수 있다.  
# 이럴때를 대비해서 Out-Of-Vocabulary OOV의 정수 인코딩 값을 사전에 추가해둔다.
word_to_index['OOV'] = len(word_to_index) + 1

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [9]:
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


### Counter를 사용하여 구현하기

In [10]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


sentences = []
stop_words = set(stopwords.words("english"))

for i in text:
		# 문장 토큰화
    sentence = word_tokenize(i)
    result = []
    for word in sentence:
				# 소문자화
        word = word.lower()
				# 불용어 제거
        if word not in stop_words:
            if len(word) > 2:
                result.append(word)
    sentences.append(result)
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [11]:
# from nltk import FreqDist

# 문장에 속한 단어들을 한 리스트로 합친다.
words = sum(sentences, [])
# words = np.hstack(sentences)

# 각 단어별로 빈도수 Dictionary를 만든다.
vocab = Counter(words)
# vocab = FreqDist(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [12]:
# 상위 5개의 단어만 저장하는 코드
vocab_size = 5
vocab = vocab.most_common(vocab_size)
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [13]:
# 정수 인덱스 부여
word_to_index = {word: i+1 for i, (word, freq) in enumerate(vocab)}
word_to_index['OOV'] = len(vocab) + 1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [14]:
# 정수 인코딩
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


### Keras의 텍스트 전처리 이용하기

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 입력한 코퍼스의 단어들을 빈도수 기준으로 모음 (Counter랑 비슷하게?)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

# 정수 인덱싱 작업 결과 보기
print("정수 인덱싱 결과:\n", tokenizer.word_index)

# 단어별 빈도수 보기
print("\n단어별 빈도수:\n", tokenizer.word_counts)


정수 인덱싱 결과:
 {'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}

단어별 빈도수:
 OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [16]:
# 입력 코퍼스를 정수 인코딩하여 반환 (위에서 word_to_index랑 동일)
word_to_index = tokenizer.texts_to_sequences(sentences)
print(word_to_index)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [17]:
# num_words를 넣어주면 빈도수 상위 N개만 추출 가능하다.
# OOV는 정수로 바꿀 때 아예 지워버리게 된다.
# 살리고 싶으면 oov_token을 주자! (기본 인덱스는 1)
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 2, oov_token = 'OOV')
tokenizer.fit_on_texts(sentences)
print("OOV 인덱스:", tokenizer.word_index['OOV'])
print("정수 인코딩 결과:", tokenizer.texts_to_sequences(sentences))

OOV 인덱스: 1
정수 인코딩 결과: [[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]
